In [1]:
import numpy as np
import scipy.sparse

# Item XVIII

Let $A_n$:
$$
A_n = 
\begin{bmatrix}
1 & -2 &  0 & \dots & 0 \\
0 &  1 & -2 & 0 & \dots \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & \ddots & 0 & 1 & -2 \\
0 & \dots & \dots & 0 & 1
\end{bmatrix} \in \mathbb{R}^{n \times n}
$$
* Determine $A_n^{-1}$
* Determine $\kappa_\infty(A_n) = ||A_n||_\infty ||A_n^{-1}||_\infty$.
* Solve the largest linear system of equations you can solve in 1 minute with the solution $x$ equal to $-1_n + U (-\delta,\delta)$ using Backward Substitution for $\delta =10^{-14}$. Notice, after you generate $x$ you need to find the RHS and then solve it, hopefully, coming back to the solution you defined previously. Did you recover the solution? 
---

### Item A
We can write $A_n = I + R$ where 
$$
R = [r_{ij}]_{i \in \{1..n\}\\j \in \{1..n\}} \text{ where } r_{ij} =
\begin{cases}
-2 & \text{if $j=i+1$} \\
0 &  \text{otherwise}
\end{cases}
$$
We can see that:
$$
R^k = \left[r^{(k)}_{ij}\right]_{i \in \{1..n\} \\ j \in \{1..n\}} \text{ where } r_{ij} =
\begin{cases}
(-2)^k & \text{if $j=i+k$} \\
0 &  \text{otherwise}
\end{cases}
$$

And we make use of the identity [1]:
$$
(I + R)^{-1} = I + \sum_{k=1}^{n-1} (-1)^k R^{k}
$$

And thus we have:
\begin{align}
A_n^{-1} &= \begin{bmatrix}
1 & 2 & 4 & 8 & 16 & \dots & \\
0 & 1 & 2 & 4 & 8 & \dots & \\
\vdots & \ddots & \ddots & \ddots & \ddots \\
0 & \dots & \dots & \dots & \dots & 1 \\
\end{bmatrix}
\\&= [a^{(-1)}_{ij}]_{i \in \{1..n\}\\ j \in \{1..n\}} \text{ where } a^{(-1)}_{ij} =
\begin{cases}
2^{j-i} & \text{if $j \geq i$} \\
0 & \text{otherwise}
\end{cases}
\end{align}

### Item B

The definition of the norm $||A||_\infty$ is:
$$
\sup_x \frac{||Ax||_\infty}{||x||_\infty}
$$
In order to calculate $||A||_\infty$, let $w$ be the vector so that:
$$
\sup_x \frac{||Ax||_\infty}{||x||_\infty} = \frac{||Aw||_\infty}{||w||_\infty} \quad \wedge \quad ||w||_\infty=1
$$
We have that
$$
||Aw||_\infty = \max\left(\max(w_i -2 w_{i+1} )_{i \in \{1..n{-}1\}},w_{n}\right)
$$
this can be maximized when $w_i=1$ and $w_{i+1}=-1$ for any $i<n$.

So, we can pick the vector:
$$
w = [1,-1,0,\dots,0]
$$
that follows the conditions.

And we have that $\frac{||Aw||_\infty}{||w||_\infty} = ||A||_\infty = 3$.


For $||A^{-1}_n||$ we can see that the first component of $Aw$ will have the largest absolute value if each $w_i$ has the same sign (as $||A^{-1}_n w||$ would be smaller if this doesn't hold), so we pick $w = [1,1,1,\dots,1]$, as it maximizes:
$$
||Aw||_\infty = \sum_{k=0}^{n-1} 2^k w_i
$$
while ensuring $||w||_\infty=1$.

This results in
$$
||A^{-1}_n w|| = \sum_{k=0}^{n-1} 2^k w_i = \sum_{k=0}^{n-1} 2^k = 2^n-1 = ||A^{-1}_n|| \,.
$$

Finally $\kappa_\infty(A_n) = ||A_n||_\infty ||A_n^{-1}||_\infty = 3 \cdot (2^{n}-1)$

### Item C

Assuming a general implementation of backward sustitution is requested (it will be more efficient to just work with the cells on the extended diagonal).

In [2]:
def problem(n,theta=1e-14):
    sol = -1.0+(2*np.random.random(n)-1.0)*theta
    matrix_a = scipy.sparse.eye(n,dtype='int',format="lil")
    j_s = range(1,n)
    i_s = range(0,n-1)
    matrix_a[(i_s,j_s)] = -2
    return matrix_a,sol

In [3]:
def backward_subs(matrix,b):
    n = matrix.shape[0]
    xs = np.zeros(n)
    for i in range(n-1,-1,-1):
        col = matrix[i,i+1:].toarray()
        xs[i] = b[i] - np.sum(xs[i+1:]*col)
        xs[i] /= matrix[i,i]
    return xs

In [4]:
aa,bb = problem(10)
print(aa)
print(np.array(aa[2,3:]))
print(bb)

  (0, 0)	1
  (0, 1)	-2
  (1, 1)	1
  (1, 2)	-2
  (2, 2)	1
  (2, 3)	-2
  (3, 3)	1
  (3, 4)	-2
  (4, 4)	1
  (4, 5)	-2
  (5, 5)	1
  (5, 6)	-2
  (6, 6)	1
  (6, 7)	-2
  (7, 7)	1
  (7, 8)	-2
  (8, 8)	1
  (8, 9)	-2
  (9, 9)	1
  (0, 0)	-2
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


In [5]:
xx = backward_subs(aa,bb)
recovered_bb = aa.dot(xx)
print("bb")
print(bb)
print("recovered_bb")
print(recovered_bb)

bb
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
recovered_bb
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


We generate problems up to $n=100$. Larger problems weren't generated because the powers of 2 do overflow (altough we could make problems of a size large enough to that the Backpropagation takes 1min, some of the values on it will be $\text{nan}$.

In [15]:
# Now generate larger problem
SIZES = [10,100,1000]
AAs = {}
BBs = {}
XXs = {}

for siz in SIZES:
    print("Backward substitution on size %d:"%siz)
    aa,bb = problem(siz)
    AAs[siz] = aa
    BBs[siz] = bb
    %time xx = backward_subs(aa,bb)
    XXs[siz] = xx

Backward substitution on size 10:
CPU times: user 2.11 ms, sys: 0 ns, total: 2.11 ms
Wall time: 2.12 ms
Backward substitution on size 100:
CPU times: user 9.78 ms, sys: 0 ns, total: 9.78 ms
Wall time: 9.57 ms
Backward substitution on size 1000:
CPU times: user 96 ms, sys: 66 µs, total: 96.1 ms
Wall time: 89.4 ms


In [22]:
for siz in SIZES:
    aa = AAs[siz]
    bb = BBs[siz]
    xx = XXs[siz]
    recovered_bb = aa.dot(xx)
    print("For n=%d"%siz)
    print("    bb")
    print(recovered_bb)
    print("    Max error: %f"%(np.max(np.abs(recovered_bb-bb))))

For n=10
    bb
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
    Max error: 0.000000
For n=100
    bb
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -2. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
    Max error: 1.000000
For n=1000
    bb
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  

We can se that the recovered $b$ only has value $-1$ in the last components, and the previous ones cannot be reconstructed. It can also be seen that $10^{-14}$ is a variation too small and it's absorved by the $-1$.

### References

* [1] https://math.stackexchange.com/a/47554